In [1]:
from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin"
)
cur = conn.cursor()

# 1) Create Silver schema inside Delta catalog
cur.execute("""
CREATE SCHEMA IF NOT EXISTS delta.hyb_silver
""")
print("Silver schema ready.")

# 2) Create Silver table structure
cur.execute("""
CREATE TABLE IF NOT EXISTS delta.hyb_silver.ecg_silver (
    timestamp BIGINT,
    device_name VARCHAR,
    ecg DOUBLE,
    annotation VARCHAR
)
""")
print("Silver table ready.")

# 3) Show schema
cur.execute("DESCRIBE delta.hyb_silver.ecg_silver")
schema = cur.fetchall()

for col in schema:
    print(col)

Silver schema ready.
Silver table ready.
['timestamp', 'bigint', '', '']
['device_name', 'varchar', '', '']
['ecg', 'double', '', '']
['annotation', 'varchar', '', '']


In [2]:
from pyspark.sql import SparkSession
from trino.dbapi import connect

# ==========================
# Spark session
# ==========================
spark = SparkSession.builder \
    .appName("Delta + S3 + Trino Pipeline") \
    .config("spark.jars.packages", "io.delta:delta-core_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .getOrCreate()

raw_path = "s3a://warehouse/fs/ecg_raw/"
archive_base = "s3a://warehouse/fs/ecg_archive/"

# ==========================================
# Infer schema statically (REQUIRED)
# ==========================================
static_df = spark.read.parquet(raw_path)
schema = static_df.schema
print("Inferred schema:")
print(schema)


# ==========================================
# Move raw files after writing to Trino
# ==========================================
def move_raw_files():
    hconf = spark._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hconf)

    src = spark._jvm.org.apache.hadoop.fs.Path(raw_path)
    dst = spark._jvm.org.apache.hadoop.fs.Path(archive_base)

    if not fs.exists(dst):
        fs.mkdirs(dst)

    files = fs.listStatus(src)
    for f in files:
        oldp = f.getPath()
        newp = spark._jvm.org.apache.hadoop.fs.Path(dst, oldp.getName())
        fs.rename(oldp, newp)
        print("Moved:", oldp, "→", newp)


# ==========================================
# foreachBatch → write each microbatch to Trino
# ==========================================
def write_to_trino(batch_df, batch_id):

    if batch_df.count() == 0:
        return

    print(f"Processing batch {batch_id}...")

    rows = [tuple(r) for r in batch_df.collect()]

    conn = connect(
        host="localhost",
        port=8082,
        user="admin",
        catalog="delta",
        schema="hyb_silver"
    )
    cur = conn.cursor()

    batch_size = 500

    for i in range(0, len(rows), batch_size):
        batch = rows[i:i + batch_size]

        values_sql = ", ".join(
            f"({r[0]}, '{r[1]}', {r[2]}, {'NULL' if r[3] is None else repr(r[3])})"
            for r in batch
        )

        insert_sql = f"""
        INSERT INTO ecg_silver (timestamp, device_name, ecg, annotation)
        VALUES {values_sql}
        """

        cur.execute(insert_sql)
        conn.commit()

        print(f"Inserted {len(batch)} rows into Trino Silver.")

    move_raw_files()
    print(f"Batch {batch_id} archived.")


# ==========================================
# Streaming read from S3
# ==========================================
stream_df = spark.readStream \
    .format("parquet") \
    .schema(schema) \
    .load(raw_path)

query = stream_df.writeStream \
    .foreachBatch(write_to_trino) \
    .option("checkpointLocation", "s3a://warehouse/checkpoints/ecg_stream/") \
    .start()

query.awaitTermination()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/09 04:15:35 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/09 04:15:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-core_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-90ee7ba7-e674-46cd-8e11-d42c76378022;1.0
	confs: [default]
:: resolution report :: resolve 2630ms :: artifacts dl 0ms
	:: modules in use:
	---------------------------------------------------------------------
	|                  |            modules           

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("Delta S3 Notebook")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

        # MinIO S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        # Warehouse
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")

        # Hive Metastore (only works if you exposed port 9083 from docker)
        .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
        .config("spark.sql.catalogImplementation", "hive")

        .getOrCreate()
)

print("Spark session started:", spark)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/10 05:45:21 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/10 05:45:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0db8bc3-e5c0-4e1c-a0e6-8d3fd83c393f;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	f

Spark session started: <pyspark.sql.session.SparkSession object at 0x70bfca2598a0>


In [4]:
df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")
df.show()

25/12/10 05:45:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually. SQLSTATE: 42KD9

In [17]:
spark.sql("DROP TABLE IF EXISTS hyb_bronze.ecg_test")

DataFrame[]

In [29]:
# Force overwrite the folder and initialize it as an empty Delta table
(
    df.limit(0)
      .write.format("delta")
      .mode("overwrite")
      .option("overwriteSchema", "true")
      .save("s3a://warehouse/hyb_bronze.db/ecg_test")
)
print("Folder reset successfully")


Folder reset successfully


In [30]:
spark.sql("""
DROP TABLE IF EXISTS hyb_bronze.ecg_test
""")

spark.sql("""
CREATE TABLE hyb_bronze.ecg_test (
    timestamp STRING,
    device_name STRING,
    ecg STRING,
    annotation STRING
)
USING delta
LOCATION 's3a://warehouse/hyb_bronze.db/ecg_test'
""")

print("Table hyb_bronze.ecg_test created")

25/12/09 04:38:23 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`hyb_bronze`.`ecg_test` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


Table hyb_bronze.ecg_test created


In [47]:
spark.sql("ALTER TABLE hyb_bronze.ecg_test ADD COLUMNS (day STRING)")

DataFrame[]

In [48]:
spark.sql("SHOW TABLES IN hyb_bronze").show(truncate=False)

+----------+-------------+-----------+
|namespace |tableName    |isTemporary|
+----------+-------------+-----------+
|hyb_bronze|ecg_bronze   |false      |
|hyb_bronze|ecg_test     |false      |
|hyb_bronze|sample_bronze|false      |
|hyb_bronze|test_table   |false      |
+----------+-------------+-----------+



In [49]:
spark.sql("select * from hyb_bronze.ecg_test").show()

+-------------+-----------+-----+----------+----+
|    timestamp|device_name|  ecg|annotation| day|
+-------------+-----------+-----+----------+----+
|1765023734338|  Person_01| 0.05|      NULL|NULL|
|1765023734343|  Person_01| 0.15|      NULL|NULL|
|1765023734349|  Person_01| 0.17|      NULL|NULL|
|1765023734354|  Person_01|0.125|      NULL|NULL|
|1765023734363|  Person_01|0.145|      NULL|NULL|
|1765023734369|  Person_01| 0.12|      NULL|NULL|
|1765023734374|  Person_01| 0.22|      NULL|NULL|
|1765023734379|  Person_01| 0.28|      NULL|NULL|
|1765023734385|  Person_01|0.275|      NULL|NULL|
|1765023734391|  Person_01|0.155|      NULL|NULL|
|1765023734397|  Person_01|0.125|      NULL|NULL|
|1765023734404|  Person_01|0.265|      NULL|NULL|
|1765023734410|  Person_01| 0.28|      NULL|NULL|
|1765023734415|  Person_01| 0.24|      NULL|NULL|
|1765023734421|  Person_01| 0.07|      NULL|NULL|
|1765023734427|  Person_01|0.035|      NULL|NULL|
|1765023734433|  Person_01|0.255|      NULL|NULL|


In [45]:
from py4j.java_gateway import java_import

ingest_date = "20251206"

src_path = f"s3a://warehouse/fs/ecg_raw/day={ingest_date}"
dst_path = f"s3a://warehouse/fs/archive_ecg_raw/day={ingest_date}"

sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()

# Import java.net.URI
java_import(sc._jvm, "java.net.URI")

# Get filesystem PROPERLY for S3A
fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI(src_path), 
    hadoop_conf
)

src = sc._jvm.org.apache.hadoop.fs.Path(src_path)
dst = sc._jvm.org.apache.hadoop.fs.Path(dst_path)

# Create archive folder
fs.mkdirs(dst)

# List only files
files = fs.listStatus(src)
files = [f.getPath().toString() for f in files if f.isFile()]

print("Files:", files)


Files: ['s3a://warehouse/fs/ecg_raw/day=20251206/t_20251206_130843_058_u_3ce3c1a3-5ddb-406f-ac57-818c824c7afb.parquet', 's3a://warehouse/fs/ecg_raw/day=20251206/t_20251206_130843_058_u_8972c51c-5dc2-4d75-8e63-335a856d8d49.parquet']


In [50]:
from pyspark.sql.functions import lit

target_table = "hyb_bronze.ecg_test"

for file in files:
    print(f"Processing file: {file}")

    # Read 1 parquet file
    df = spark.read.parquet(file)

    # Add the date partition column
    df = df.withColumn("day", lit(ingest_date))

    # Write to Delta (append, partitioned)
    df.write \
      .format("delta") \
      .mode("append") \
      .saveAsTable(target_table)

print("All files written to Delta.")

Processing file: s3a://warehouse/fs/ecg_raw/day=20251206/t_20251206_130843_058_u_3ce3c1a3-5ddb-406f-ac57-818c824c7afb.parquet
Processing file: s3a://warehouse/fs/ecg_raw/day=20251206/t_20251206_130843_058_u_8972c51c-5dc2-4d75-8e63-335a856d8d49.parquet
All files written to Delta.


In [51]:
for file in files:
    file_name = file.split("/")[-1]

    src_file = sc._jvm.org.apache.hadoop.fs.Path(f"{src_path}/{file_name}")
    dst_file = sc._jvm.org.apache.hadoop.fs.Path(f"{dst_path}/{file_name}")

    ok = fs.rename(src_file, dst_file)

    if ok:
        print(f"Moved {file_name} → archive")
    else:
        print(f"Failed to move {file_name} (check permissions)")

Moved t_20251206_130843_058_u_3ce3c1a3-5ddb-406f-ac57-818c824c7afb.parquet → archive
Moved t_20251206_130843_058_u_8972c51c-5dc2-4d75-8e63-335a856d8d49.parquet → archive


In [2]:
from pyspark.sql import SparkSession
from trino.dbapi import connect
from py4j.java_gateway import java_import
from pyspark.sql.functions import lit

spark = (
    SparkSession.builder
        .appName("Delta S3 Notebook")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

        # MinIO S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        # Warehouse
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")

        # Hive Metastore (only works if you exposed port 9083 from docker)
        .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
        .config("spark.sql.catalogImplementation", "hive")

        .getOrCreate()
)

print("Spark session started:", spark)

root_path = "s3a://warehouse/fs/ecg_raw"
archive_root = "s3a://warehouse/fs/archive_ecg_raw"
target_table = "hyb_bronze.ecg_test"

# prepare Hadoop FS
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
java_import(sc._jvm, "java.net.URI")

fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI(root_path), 
    hadoop_conf
)

root = sc._jvm.org.apache.hadoop.fs.Path(root_path)

# -----------------------------------
# 1. Find all day=YYYYMMDD directories
# -----------------------------------
status = fs.listStatus(root)
day_folders = [p.getPath().getName() for p in status if p.isDirectory()]

print("Found day folders:", day_folders)

# -----------------------------------
# 2. Loop through each day folder
# -----------------------------------
for day_folder in day_folders:

    ingest_date = day_folder.replace("day=", "")
    print(f"\nProcessing day {ingest_date}")

    src_path = f"{root_path}/{day_folder}"
    dst_path = f"{archive_root}/{day_folder}"

    src = sc._jvm.org.apache.hadoop.fs.Path(src_path)
    dst = sc._jvm.org.apache.hadoop.fs.Path(dst_path)

    # create archive folder
    fs.mkdirs(dst)

    # list files only
    files = fs.listStatus(src)
    files = [f.getPath().toString() for f in files if f.isFile()]

    print("Files:", files)

    # -----------------------------------
    # 3. Write to Delta table
    # -----------------------------------
    for file in files:
        print(f"  Writing file: {file}")

        df = spark.read.parquet(file)
        df = df.withColumn("day", lit(ingest_date))

        df.write \
          .format("delta") \
          .mode("append") \
          .saveAsTable(target_table)

    # -----------------------------------
    # 4. Move files to archive
    # -----------------------------------
    for file in files:
        file_name = file.split("/")[-1]

        src_file = sc._jvm.org.apache.hadoop.fs.Path(f"{src_path}/{file_name}")
        dst_file = sc._jvm.org.apache.hadoop.fs.Path(f"{dst_path}/{file_name}")

        ok = fs.rename(src_file, dst_file)

        if ok:
            print(f"  Moved {file_name} → archive")
        else:
            print(f"  FAILED moving {file_name}")

print("\nAll days processed successfully.")


Spark session started: <pyspark.sql.session.SparkSession object at 0x7499633b51b0>
Found day folders: ['day=20251206', 'day=20251207', 'day=20251210']

Processing day 20251206
Files: []

Processing day 20251207
Files: []

Processing day 20251210
Files: ['s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224419_608_u_a6cbe6f2-8afb-46d7-bad4-763c7d497fe2.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224419_608_u_e036835a-d59f-4079-827a-1bb5ddabaf9b.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224720_672_u_a180d3b6-e365-4d0f-9190-73beb39d7bd9.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224820_956_u_5f48e6f8-07f0-4b08-bbb8-355bab079ad3.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224820_956_u_96900079-954e-49cf-877e-25ae5d3bd62b.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224921_972_u_cd78f322-01b3-4296-a847-3343edd90bcc.parquet', 's3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_224923_036_u_26b87a22-

AnalysisException: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 3762304f-465e-4ebc-990e-76046b18ad7f).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- timestamp: string (nullable = true)
-- device_name: string (nullable = true)
-- ecg: string (nullable = true)
-- annotation: string (nullable = true)
-- day: string (nullable = true)


Data schema:
root
-- raw_json: string (nullable = true)
-- day: string (nullable = true)

         

In [1]:
from pyspark.sql import SparkSession
from py4j.java_gateway import java_import
from pyspark.sql.functions import lit
import time

# ------------------------------------------------
# Spark session
# ------------------------------------------------
spark = (
    SparkSession.builder
        .appName("ECG Continuous Ingestion")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.databricks.delta.schema.autoMerge.enabled", "true")

        # MinIO
        .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
        .config("spark.sql.catalogImplementation", "hive")

        .getOrCreate()
)

print("Spark started")

# ------------------------------------------------
# S3 paths
# ------------------------------------------------
root_path = "s3a://warehouse/fs/ecg_raw"
archive_root = "s3a://warehouse/fs/archive_ecg_raw"
target_table = "hyb_bronze.ecg_test"

# Hadoop FS
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
java_import(sc._jvm, "java.net.URI")

fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI(root_path),
    hadoop_conf
)

root = sc._jvm.org.apache.hadoop.fs.Path(root_path)

print("Starting continuous ingestion loop...")

# ------------------------------------------------
# Continuous loop
# ------------------------------------------------
while True:
    try:
        # List day folders
        status = fs.listStatus(root)
        day_folders = [p.getPath().getName() for p in status if p.isDirectory()]

        if not day_folders:
            print("No day folders found. Waiting...")
            time.sleep(5)
            continue

        # Process each day folder
        for day_folder in day_folders:
            ingest_date = day_folder.replace("day=", "")
            print(f"\nChecking day={ingest_date}")

            src_path = f"{root_path}/{day_folder}"
            dst_path = f"{archive_root}/{day_folder}"

            src = sc._jvm.org.apache.hadoop.fs.Path(src_path)
            dst = sc._jvm.org.apache.hadoop.fs.Path(dst_path)
            fs.mkdirs(dst)

            # List files
            files = fs.listStatus(src)
            files = [f.getPath().toString() for f in files if f.isFile()]

            if not files:
                print(f"No files in {day_folder}, waiting...")
                continue

            # Process each file
            for file in files:
                print(f"Processing: {file}")

                try:
                    df = spark.read.parquet(file)
                    df = df.withColumn("day", lit(ingest_date))

                    df.write \
                        .format("delta") \
                        .option("mergeSchema", "true") \
                        .mode("append") \
                        .saveAsTable(target_table)

                    print("Write OK")

                except Exception as e:
                    print("Write ERROR:", e)

                # Archive file no matter what
                file_name = file.split("/")[-1]

                src_file = sc._jvm.org.apache.hadoop.fs.Path(f"{src_path}/{file_name}")
                dst_file = sc._jvm.org.apache.hadoop.fs.Path(f"{dst_path}/{file_name}")

                moved = fs.rename(src_file, dst_file)

                if moved:
                    print(f"Archived: {file_name}")
                else:
                    print(f"FAILED to archive {file_name}")

        time.sleep(5)

    except Exception as loop_err:
        print("Loop error:", loop_err)
        time.sleep(5)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/11 06:47:17 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/11 06:47:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-15df9fb1-775f-4de4-af1f-e0b2e8f2bb76;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	f

Spark started


25/12/11 06:47:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Starting continuous ingestion loop...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
Processing: s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_234700_320_u_2737c4d0-4eb9-4e78-b506-4afc636eb7af.parquet


25/12/11 06:47:40 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to hyb_bronze.db/ecg_test/part-00000-61aeb889-933d-42c6-9030-b62a0497b6ab-c000.snappy.parquet. This is Unsupported


Write OK
Archived: t_20251210_234700_320_u_2737c4d0-4eb9-4e78-b506-4afc636eb7af.parquet
Processing: s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_234722_493_u_29c47aca-bc94-43f9-8f8b-e14b3a6d0761.parquet
Write OK
Archived: t_20251210_234722_493_u_29c47aca-bc94-43f9-8f8b-e14b3a6d0761.parquet
Processing: s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_234722_515_u_8b4b4bbd-ae91-492b-8739-a150421eaa79.parquet
Write OK
Archived: t_20251210_234722_515_u_8b4b4bbd-ae91-492b-8739-a150421eaa79.parquet
Processing: s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_234723_478_u_0e80913c-9266-4b36-bd39-6c631b3a74d9.parquet
Write OK
Archived: t_20251210_234723_478_u_0e80913c-9266-4b36-bd39-6c631b3a74d9.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=202512

25/12/11 06:48:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Write OK
Archived: t_20251210_234822_720_u_df93e354-f61e-45f5-97ae-7692bedcdc87.parquet
Processing: s3a://warehouse/fs/ecg_raw/day=20251210/t_20251210_234822_751_u_e0ea1fb6-dd76-439d-ac66-4efe882ccf4c.parquet
Write OK
Archived: t_20251210_234822_751_u_e0ea1fb6-dd76-439d-ac66-4efe882ccf4c.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=2025

In [1]:
from pyspark.sql import SparkSession
from py4j.java_gateway import java_import
from pyspark.sql.functions import lit, current_timestamp
import time

# ------------------------------------------------
# Spark session
# ------------------------------------------------
spark = (
    SparkSession.builder
        .appName("ECG Continuous Ingestion")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.databricks.delta.schema.autoMerge.enabled", "true")

        # MinIO
        .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
        .config("spark.sql.catalogImplementation", "hive")

        .getOrCreate()
)

print("Spark started")

# ------------------------------------------------
# S3 paths
# ------------------------------------------------
root_path = "s3a://warehouse/fs/ecg_raw"
archive_root = "s3a://warehouse/fs/archive_ecg_raw"

# CHANGE: target table -> SILVER
target_table = "hyb_silver.ecg_silver"

# Hadoop FS
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
java_import(sc._jvm, "java.net.URI")

fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI(root_path),
    hadoop_conf
)

root = sc._jvm.org.apache.hadoop.fs.Path(root_path)

print("Starting continuous ingestion loop...")

# ------------------------------------------------
# Continuous loop
# ------------------------------------------------
while True:
    try:
        # List day folders
        status = fs.listStatus(root)
        day_folders = [p.getPath().getName() for p in status if p.isDirectory()]

        if not day_folders:
            print("No day folders found. Waiting...")
            time.sleep(5)
            continue

        # Process each day folder
        for day_folder in day_folders:
            ingest_date = day_folder.replace("day=", "")
            print(f"\nChecking day={ingest_date}")

            src_path = f"{root_path}/{day_folder}"
            dst_path = f"{archive_root}/{day_folder}"

            src = sc._jvm.org.apache.hadoop.fs.Path(src_path)
            dst = sc._jvm.org.apache.hadoop.fs.Path(dst_path)
            fs.mkdirs(dst)

            # List files
            files = fs.listStatus(src)
            files = [f.getPath().toString() for f in files if f.isFile()]

            if not files:
                print(f"No files in {day_folder}, waiting...")
                continue

            # Process each file
            for file in files:
                print(f"Processing: {file}")

                try:
                    df = spark.read.parquet(file)

                    # ADD Silver metadata
                    df = (
                        df
                        .withColumn("day", lit(ingest_date))
                        .withColumn("ingest_time", current_timestamp())
                    )

                    df.write \
                        .format("delta") \
                        .option("mergeSchema", "true") \
                        .mode("append") \
                        .saveAsTable(target_table)

                    print("Write OK")

                except Exception as e:
                    print("Write ERROR:", e)

                # Archive file no matter what
                file_name = file.split("/")[-1]

                src_file = sc._jvm.org.apache.hadoop.fs.Path(f"{src_path}/{file_name}")
                dst_file = sc._jvm.org.apache.hadoop.fs.Path(f"{dst_path}/{file_name}")

                moved = fs.rename(src_file, dst_file)

                if moved:
                    print(f"Archived: {file_name}")
                else:
                    print(f"FAILED to archive {file_name}")

        time.sleep(5)

    except Exception as loop_err:
        print("Loop error:", loop_err)
        time.sleep(5)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/15 19:18:08 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/15 19:18:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6fecac5a-443c-4836-9b16-0c5be78f48f6;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	f

Spark started


25/12/15 19:18:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Starting continuous ingestion loop...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251206
No files in day=20251206, waiting...

C

25/12/15 19:22:08 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to hyb_silver.db/ecg_silver/part-00000-5214e9a7-705e-47d7-9612-9054bb725ed2-c000.snappy.parquet. This is Unsupported
25/12/15 19:22:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


...

Write OK
Archived: t_20251215_122317_037_u_6fce9cde-3553-4e0b-bc77-bdb613ac4668.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking da

Write OK
Archived: t_20251215_122413_797_u_e6ef86a3-b18e-4a2f-a01d-59e1f0d1021b.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
Processing: s3a://warehouse/fs/ecg_raw/day=20251215/t_20251215_122416_229_u_ffeab085-5276-4043-85f5-59cc9572c4df.parquet


Write OK
Archived: t_20251215_122416_229_u_ffeab085-5276-4043-85f5-59cc9572c4df.parquet
Processing: s3a://warehouse/fs/ecg_raw/day=20251215/t_20251215_122417_689_u_a283ce1b-9241-4307-b420-ad71bcebf183.parquet


Write OK
Archived: t_20251215_122417_689_u_a283ce1b-9241-4307-b420-ad71bcebf183.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking da

Write OK
Archived: t_20251215_122513_634_u_223a205b-83df-43fc-8920-1745ed5aa7ce.parquet

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking day=20251215
No files in day=20251215, waiting...

Checking day=20251206
No files in day=20251206, waiting...

Checking day=20251207
No files in day=20251207, waiting...

Checking day=20251210
No files in day=20251210, waiting...

Checking da